In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import time
import yaml
from types import SimpleNamespace

import sys, os
from pathlib import Path

# 1) figure out your project root (one level up from this notebook)
project_root = Path().resolve().parent

# 2) add it to the front of sys.path
sys.path.insert(0, str(project_root))

from src.data.dataloader import EML_Dataset
from src.models.moclip import build_moclip
from hydra import initialize, compose

import tqdm as tqdm
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#Dataloader
root = '/home/rodionsa/Data/EML_Datasets/EML_466k_dataset_bias.npz'
Dataset = EML_Dataset(root)

total_len = len(Dataset)
train_len = int(0.8 * total_len)
val_len = int(0.1 * total_len)
test_len = total_len - train_len - val_len

train_ds, val_ds, test_ds = torch.utils.data.random_split(
    Dataset, [train_len, val_len, test_len],
    generator = torch.Generator().manual_seed(42)
)


In [3]:
#Use dataloader
train_loader = DataLoader(train_ds, batch_size=3000, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=3000, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=3000, shuffle=False)

dataset_loader = DataLoader(Dataset, batch_size=3000, shuffle=False)

In [2]:
#Donwload MOCLIP
# Load YAML config file
with open('../src/models/config.yaml', 'r') as file:
    cfg_dict = yaml.safe_load(file)

# Convert to SimpleNamespace for easy attribute access
cfg = SimpleNamespace(**cfg_dict)

moclip = build_moclip(cfg)
moclip = moclip.to(device)